In [ ]:
!pip install -q efficientnet

In [ ]:
!pip install -q chart_studio

In [ ]:
# import modules

import numpy as np 
import pandas as pd 
import math
import os
import glob
import random
import matplotlib.pyplot as plt
import seaborn as sns
import PIL

# plotly
import plotly.express as px
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot
import cufflinks
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')

from matplotlib.image import imread
import cv2

from sklearn.model_selection import ( KFold, train_test_split, 
                                    cross_validate, cross_val_score, GridSearchCV )
from sklearn.metrics import ( roc_curve, auc, precision_recall_curve, 
                             average_precision_score, ConfusionMatrixDisplay )
from mlxtend.plotting import plot_confusion_matrix

from random import choices
from functools import partial
import re
from kaggle_datasets import KaggleDatasets

# tensorflow
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.layers import (Dense, Flatten, Dropout, LSTM, 
                                     Bidirectional, Lambda, Reshape,
                                    GlobalAveragePooling2D) 
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras import optimizers
from keras.utils.vis_utils import plot_model

import efficientnet.tfkeras as efn

# Suppress warnings 
import warnings
warnings.simplefilter(action='ignore')

In [ ]:
%matplotlib inline
sns.set(context="paper", font="monospace")
sns.set(style="whitegrid")
plt.style.use('fivethirtyeight')

## 1. Data Loading and Exploration

In [ ]:
# available files and folders 
print(os.listdir("../input/siim-isic-melanoma-classification"))

In [ ]:
train = pd.read_csv('/kaggle/input/siim-isic-melanoma-classification/train.csv')

test = pd.read_csv('/kaggle/input/siim-isic-melanoma-classification/test.csv')

print('Training data shape: ', train.shape)
train.head() 

In [ ]:
# Train and test data info
print('Train Set')
print(train.info())
print('-------------')
print('Test Set')
print(test.info())

#### Explore melanoma target

In [ ]:
train['benign_malignant'].value_counts()

In [ ]:
train['benign_malignant'].value_counts(normalize=True).iplot(kind='bar',
                                                      yTitle='Percentage', 
                                                      linecolor='black', 
                                                      opacity=0.7,
                                                      color='red',
                                                      theme='pearl',
                                                      bargap=0.2,
                                                      gridcolor='white',
                                                      title='Melanoma Target Distribution')

Highly imbalanced target. There are significantly more benign images compared to malignant images in the train dataset.

#### Sex distribution

In [ ]:
train['sex'].value_counts(normalize=True)

In [ ]:
train['sex'].value_counts(normalize=True).iplot(kind='bar',
                                                yTitle='Percentage', 
                                                linecolor='black', 
                                                opacity=0.7,
                                                color='green',
                                                theme='pearl',
                                                bargap=0.3,
                                                gridcolor='white',
                                                title='Sex Distribution')

#### Gender vs Target

In [ ]:
z = train.groupby(['target','sex'])['benign_malignant'].count().to_frame().reset_index()
z.style.background_gradient(cmap='Reds')  

In [ ]:
sns.catplot(x='target',y='benign_malignant', hue='sex',data=z,kind='bar')
plt.ylabel('Count')
plt.xlabel('benign:0 vs malignant:1');

#### Location of imaged site

In [ ]:
train['anatom_site_general_challenge'].value_counts(normalize=True).sort_values()

In [ ]:
train['anatom_site_general_challenge'].value_counts(normalize=True).sort_values().iplot(kind='barh',
                                                      xTitle='Percentage', 
                                                      linecolor='black', 
                                                      opacity=0.7,
                                                      color='#FB8072',
                                                      theme='pearl',
                                                      bargap=0.2,
                                                      gridcolor='white',
                                                      title='Anatomical Site Distribution')

#### Age distribution of patients

In [ ]:
train['age_approx'].iplot(kind='hist',bins=30,color='orange',xTitle='Age distribution',yTitle='Count')

#### Diagnosis Distribution

In [ ]:
train['diagnosis'].value_counts()

In [ ]:
train['diagnosis'].value_counts(normalize=True).sort_values().iplot(kind='barh',
                                                      xTitle='Percentage', 
                                                      linecolor='black', 
                                                      opacity=0.7,
                                                      color='blue',
                                                      theme='pearl',
                                                      bargap=0.2,
                                                      gridcolor='white',
                                                      title='Diagnosis Distribution')

#### Visualizing images

In [ ]:
# Function for plotting samples
def plot_samples(samples):  
    fig, axes = plt.subplots(nrows=4, ncols=5, figsize=(32,16))
    for i in range(len(samples)):
        image = imread(samples[i])
        ax = axes[i//5][i%5]
        ax.imshow(image)
        if i<10: # first 10 files
            ax.set_title("Benign", fontsize=20)
        else:
            ax.set_title("Malignant", fontsize=20)
        ax.axis('off')

In [ ]:
# sample images

dirname = '/kaggle/input/siim-isic-melanoma-classification/jpeg/train/'
sample_img = []

# get 10 benign image files 
benign_ = train[train['benign_malignant'] == 'benign']['image_name'][:10]

# get 10 malignant image files 
malignant_ = train[train['benign_malignant'] == 'malignant']['image_name'][:10]

# get benign filepaths
for i in benign_:
    sample_img.append(dirname + i + '.jpg')

# get malignant filepaths 
for j in malignant_:
    sample_img.append(dirname + j + '.jpg')

plot_samples(sample_img)
plt.suptitle('Melanoma Samples', fontsize=30)
# plt.tight_layout()
plt.show()

## 2. Data Preparation and Augmentation

#### Connect to TPU

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE

# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is set. 
    # On Kaggle this is always the case.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy() 

REPLICAS = strategy.num_replicas_in_sync
print("REPLICAS: ", strategy.num_replicas_in_sync)

#### Set parameters

In [ ]:
SEED = 42
FOLDS = 3
EFF_NETS = 6 
BATCH_SIZES = [bs * strategy.num_replicas_in_sync for bs in [32]*FOLDS] 
IMG_SIZES = [512]*FOLDS
EPOCHS = 10 
LR = 0.00004
LABEL_SMOOTHING = 0.05
TTA = 15 # test time augmentation

In [ ]:
# Seed to make sure the same random numbers are generated on multiple executions
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

#### Get Dataset

In [ ]:
# Get file paths for train, validation and test

GCS_PATH = KaggleDatasets().get_gcs_path('512x512-melanoma-tfrecords-70k-images')
GCS_PATH2 = KaggleDatasets().get_gcs_path('isic2019-512x512')

train_filenames = tf.io.gfile.glob(GCS_PATH + '/train*.tfrec')
test_filenames = tf.io.gfile.glob(GCS_PATH + '/test*.tfrec')

print("# TRAINING_FILENAMES", len(train_filenames))
print("# TEST_FILENAMES", len(test_filenames))

In [ ]:
OLD_COMP_FILENAMES = tf.io.gfile.glob(GCS_PATH2 + '/*.tfrec')
print("# OLD_COMP_FILENAMES", len(OLD_COMP_FILENAMES))

#### Image loading functions
We create some functions for loading TFRecords to get the images, target and image names

* **decode_image**: to transform images to a tensor, normalize it, and reshape it into the correct shape for TPU
* **read_tfrecord**: read the TFRecord, returns the image tensor, and based on the input arguments, return the label value, image name, or nothing (0)
* **load_dataset**: reads data from the TFRecords. Here we can choose whether to shuffle the data or not. We will do that for the train, but not the validation and test dataset.
* **count_data_items**: counts the number of images in a file
* **data_augment**: performs data augmentation; techniques used to increase the amount of data by adding slightly modified copies of already existing data or newly created synthetic data from existing data
* **plot_transform**: plots some examples of augmented images

In [ ]:
ROT_ = 180.0
SHR_ = 2.0
HZOOM_ = 8.0
WZOOM_ = 8.0
HSHIFT_ = 8.0
WSHIFT_ = 8.0

def get_mat(rotation, shear, height_zoom, width_zoom, height_shift, width_shift):
    # returns 3x3 transformmatrix which transforms indicies
        
    # CONVERT DEGREES TO RADIANS
    rotation = math.pi * rotation / 180.
    shear    = math.pi * shear    / 180.

    def get_3x3_mat(lst):
        return tf.reshape(tf.concat([lst],axis=0), [3,3])
    
    # ROTATION MATRIX
    c1   = tf.math.cos(rotation)
    s1   = tf.math.sin(rotation)
    one  = tf.constant([1],dtype='float32')
    zero = tf.constant([0],dtype='float32')
    
    rotation_matrix = get_3x3_mat([c1,   s1,   zero, 
                                   -s1,  c1,   zero, 
                                   zero, zero, one])    
    # SHEAR MATRIX
    c2 = tf.math.cos(shear)
    s2 = tf.math.sin(shear)    
    
    shear_matrix = get_3x3_mat([one,  s2,   zero, 
                                zero, c2,   zero, 
                                zero, zero, one])        
    # ZOOM MATRIX
    zoom_matrix = get_3x3_mat([one/height_zoom, zero,           zero, 
                               zero,            one/width_zoom, zero, 
                               zero,            zero,           one])    
    # SHIFT MATRIX
    shift_matrix = get_3x3_mat([one,  zero, height_shift, 
                                zero, one,  width_shift, 
                                zero, zero, one])
    
    return K.dot(K.dot(rotation_matrix, shear_matrix), 
                 K.dot(zoom_matrix,     shift_matrix))


def transform(image, DIM=512):    
    # input image - is one image of size [dim,dim,3] not a batch of [b,dim,dim,3]
    # output - image randomly rotated, sheared, zoomed, and shifted
    XDIM = DIM%2 #fix for size 331
    
    rot = ROT_ * tf.random.normal([1], dtype='float32')
    shr = SHR_ * tf.random.normal([1], dtype='float32') 
    h_zoom = 1.0 + tf.random.normal([1], dtype='float32') / HZOOM_
    w_zoom = 1.0 + tf.random.normal([1], dtype='float32') / WZOOM_
    h_shift = HSHIFT_ * tf.random.normal([1], dtype='float32') 
    w_shift = WSHIFT_ * tf.random.normal([1], dtype='float32') 

    # GET TRANSFORMATION MATRIX
    m = get_mat(rot,shr,h_zoom,w_zoom,h_shift,w_shift) 

    # LIST DESTINATION PIXEL INDICES
    x   = tf.repeat(tf.range(DIM//2, -DIM//2,-1), DIM)
    y   = tf.tile(tf.range(-DIM//2, DIM//2), [DIM])
    z   = tf.ones([DIM*DIM], dtype='int32')
    idx = tf.stack( [x,y,z] )
    
    # ROTATE DESTINATION PIXELS ONTO ORIGIN PIXELS
    idx2 = K.dot(m, tf.cast(idx, dtype='float32'))
    idx2 = K.cast(idx2, dtype='int32')
    idx2 = K.clip(idx2, -DIM//2+XDIM+1, DIM//2)
    
    # FIND ORIGIN PIXEL VALUES           
    idx3 = tf.stack([DIM//2-idx2[0,], DIM//2-1+idx2[1,]])
    d    = tf.gather_nd(image, tf.transpose(idx3))
        
    return tf.reshape(d,[DIM, DIM,3])

In [ ]:
def decode_image(image):
    # decode a JPEG-encoded image to a uint8 tensor
    image = tf.image.decode_jpeg(image, channels=3) 
    # cast tensor to float32 and normalize to [0, 1] range
    image = tf.cast(image, tf.float32)/255.0 
    # explicit size needed for TPU
    image = tf.reshape(image, [*IMG_SIZES[0:2], 3]) 
    return image

def read_tfrecord(example, labeled, return_imgname=False):
    tfrecord_format = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "target": tf.io.FixedLenFeature([], tf.int64)
    } if labeled else {
        "image": tf.io.FixedLenFeature([], tf.string),
        "image_name": tf.io.FixedLenFeature([], tf.string)
    }
    example = tf.io.parse_single_example(example, tfrecord_format)
    image = decode_image(example['image'])
    # returns a dataset of (image, label) pairs if labeled=True
    if labeled:
        label = tf.cast(example['target'], tf.int32)
        return image, label
    idnum = example['image_name']
    # returns a dataset of (image, image_name) pairs if return_imgname=True
    if return_imgname:
        return image, idnum
    # else returns a dataset of (image, 0) pairs 
    return image, 0

def load_dataset(filenames, labeled=True, ordered=False, return_imgname=False):
    # Read from TFRecords. For optimal performance, reading from multiple files at once and
    # disregarding data order. Order does not matter since we will be shuffling the data anyway
    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False # disable order, increase speed
    # automatically interleaves reads from multiple files
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO) 
    # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.with_options(ignore_order) 
    dataset = dataset.map(partial(read_tfrecord, labeled=labeled, 
                                  return_imgname=return_imgname), num_parallel_calls=AUTO)
    , or (image, id) pairs if labeled=False
    return dataset

def count_data_items(filenames):
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)

def data_augment(image, label=None, seed=SEED):
    # data augmentation. Thanks to the dataset.prefetch(AUTO) statement when 
    # loading dataset (below cell), this happens essentially for free on TPU. Data pipeline
    # code is executed on the "CPU" part of the TPU while the TPU itself is 
    # computing gradients.
    image = transform(image, IMG_SIZES[0])
    #image = tf.image.rot90(image,k=np.random.randint(4)) # rotate
    image = tf.image.random_flip_left_right(image, seed=seed) # flip horizontal
    image = tf.image.random_flip_up_down(image, seed=seed) # flip vertical
    image = tf.image.random_brightness(image, max_delta=0.2) # random brightness
    image = tf.image.random_contrast(image, 0.8, 1.2) # random contrast
    image = tf.image.random_saturation(image, 0.7, 1.3) # random saturation
    
    if label is None:
        return image
    else:
        return image, label

# plot augmented images sample
def plot_transform(num_images):
    fig, ax = plt.subplots(nrows=3, ncols=num_images, figsize=(12,5))
    x = (load_dataset(train_filenames, labeled=True)
                     .shuffle(SEED)
                     .batch(BATCH_SIZES[0],drop_remainder=True)                 
                     .prefetch(AUTO)
                     .unbatch().take(5))
    images = []
    imgs=[]
    for r in range(3):
        image,_ = iter(x).next()
        images.append(image)
        for i in range(0,num_images):
            image = data_augment(image=images[r])
            imgs.append(image)
    for img in range(len(imgs)):          
        ax[img//num_images][img%num_images].imshow(imgs[img])
        ax[img//num_images][img%num_images].axis('off') 

In [ ]:
# Function for plotting images in grid
def show_dataset(thumb_size, cols, rows, ds):
    mosaic = PIL.Image.new(mode='RGB', size=(thumb_size*cols + (cols-1), 
                                             thumb_size*rows + (rows-1)))
   
    for idx, data in enumerate(iter(ds)):
        img, target_or_imgid = data
        ix  = idx % cols
        iy  = idx // cols
        img = np.clip(img.numpy() * 255, 0, 255).astype(np.uint8)
        img = PIL.Image.fromarray(img)
        img = img.resize((thumb_size, thumb_size), resample=PIL.Image.BILINEAR)
        mosaic.paste(img, (ix*thumb_size + ix, 
                           iy*thumb_size + iy))

    display(mosaic)
    
eg_ds = (load_dataset(train_filenames, labeled=True)
                     .batch(BATCH_SIZES[0],drop_remainder=True)                 
                     .prefetch(AUTO)
                     .unbatch().take(10*6))  

show_dataset(64, 10, 6, eg_ds) 

Image augmentation examples. 

In [ ]:
plot_transform(7) 

In [ ]:
print("Number of Train Files", count_data_items(train_filenames))
print("Number of Test Files", count_data_items(test_filenames)) 

## 3. Transfer Learning EffNet (CNN)

#### Learning Rate Train Schedule
This is a common train schedule for transfer learning. The learning rate starts near zero, then increases to a maximum, then decays over time. 

In [ ]:
def lrfn(epoch):
    if epoch < lr_ramp_ep:
        lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start

    elif epoch < lr_ramp_ep + lr_sus_ep:
        lr = lr_max

    else:
        lr = (lr_max - lr_min) * lr_decay**(epoch - lr_ramp_ep - lr_sus_ep) + lr_min

    return lr

def get_lr_callback(batch_size=8):
    lr_start   = 0.000005
    lr_max     = 0.000003 * batch_size
    lr_min     = 0.000001
    lr_ramp_ep = 5
    lr_sus_ep  = 0
    lr_decay   = 0.3
       
    lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=False)
    return lr_callback

In [ ]:
lr_start   = 0.000005
lr_max     = 0.000003 * BATCH_SIZES[0]
lr_min     = 0.000001
lr_ramp_ep = 5
lr_sus_ep  = 0
lr_decay   = 0.3

In [ ]:
def display_training_curves(history, name, model_name):
    '''
    Plots the training process
    '''
    fig, (ax1, ax2) = plt.subplots(2, figsize=(20,15))
    fig.suptitle(model_name, size=20)
    
    # plot AUC
    ax1.plot(np.arange(EPOCHS),history.history['auc'],'-o',
             label='Train AUC',color='#ff7f0e')
    ax1.plot(np.arange(EPOCHS),history.history['val_auc'],'-o',
             label='Val AUC',color='#1f77b4')
    x = np.argmax(history.history['val_auc']); y = np.max(history.history['val_auc'])
    xdist = plt.xlim()[1] - plt.xlim()[0]; ydist = plt.ylim()[1] - plt.ylim()[0]
    ax1.scatter(x,y,s=200,color='#1f77b4')
    ax1.text(x-0.03*xdist,y-0.05*ydist,'max auc\n%.2f'%y,size=14)
    ax1.set_ylabel('AUC',size=14); ax1.set_xlabel('Epoch',size=14)
    ax1.set_title('AUC curve')
    ax1.set_xticks(list(range(EPOCHS)))
    ax1.set_xticklabels(list(range(1, EPOCHS+1)))
    ax1.legend(loc=2)

    #  plot loss
    ax2.plot(np.arange(EPOCHS),history.history['loss'],'-o',
              label='Train Loss',color='#2ca02c')
    ax2.plot(np.arange(EPOCHS),history.history['val_loss'],'-o',
              label='Val Loss',color='#d62728')
    x = np.argmin(history.history['val_loss'] )
    y = np.min(history.history['val_loss'] )
    ydist = plt.ylim()[1] - plt.ylim()[0]
    ax2.scatter(x,y,s=200,color='#d62728')
    ax2.text(x-0.03*xdist,y+0.05*ydist,'min loss',size=14)
    ax2.set_ylabel('Loss',size=14); ax2.set_xlabel('Epoch',size=14)
    ax2.set_title('Loss Curve')
    ax2.set_xticks(list(range(EPOCHS)))
    ax2.set_xticklabels(list(range(1, EPOCHS+1)))
    ax2.legend(loc=3)
    fig.savefig(name + '.png')
    plt.show() 
    

#### Get and load train, validation and test dataset

Split files to get train and validation filenames

In [ ]:
files_train, files_valid = train_test_split(
    train_filenames, test_size = 0.20, random_state = SEED)

# add old comp data to valid set
files_train += tf.io.gfile.glob(GCS_PATH2[0] + '/train*.tfrec')
# shuffle training set
np.random.shuffle(files_train) 

In [ ]:
print("# TRAINING_FILENAMES", len(files_train))
print("# VALIDATION_FILENAMES", len(files_valid))

print("# Train Files after Splitting", count_data_items(files_train))
print("# Validation Files", count_data_items(files_valid))

Get test set filenames

In [ ]:
files_test = np.sort(
    np.array(tf.io.gfile.glob(GCS_PATH + '/test*.tfrec'))) 

Load train, validation and test dataset

In [ ]:
train_dataset = (load_dataset(files_train, labeled=True)
                     .repeat() # repeat to continue getting data for aug
                     .map(data_augment, num_parallel_calls=AUTO) # data augmentation
                     .shuffle(SEED)
                     .batch(BATCH_SIZES[0],drop_remainder=True)
                     # prefetch next batch while training (autotune prefetch buffer size)
                     .prefetch(AUTO)) 

ds_valid = (load_dataset(files_valid, labeled=True, ordered=True)                                        
                     .cache()     
                     .repeat()   # repeat for data aug during val
                     .map(data_augment, num_parallel_calls=AUTO)  # data augmentation 
                     .batch(BATCH_SIZES[0]*4)  # X4 to speed up training
                     .prefetch(AUTO))

ds_test = (load_dataset(files_test, labeled=False, ordered=True) # do not shuffle
                     .repeat()                                   # repeat for TTA
                     .map(data_augment, num_parallel_calls=AUTO) # data augmentation 
                     .batch(BATCH_SIZES[0]*4)
                     .prefetch(AUTO))

#### Train and Evaluate model

In [ ]:
# initialize storage
pred_ = []; tar_ = []; val_ = []; names_ = []
# initialize for prediction storage
preds = np.zeros((count_data_items(test_filenames),1))

In [ ]:
if tpu: tf.tpu.experimental.initialize_tpu_system(tpu)
    
# USE VERBOSE=0 for silent, VERBOSE=1 for interactive, VERBOSE=2 for commit
VERBOSE = 2

if tpu: tf.tpu.experimental.initialize_tpu_system(tpu)
    
K.clear_session()
with strategy.scope():
    model = tf.keras.Sequential([
        efn.EfficientNetB6(input_shape=(IMG_SIZES[0],IMG_SIZES[0], 3),
                           weights='imagenet',include_top=False),
        GlobalAveragePooling2D(),
        # add fully connected layer, with sigmoid activation since only 2 categories
        Dense(1, activation='sigmoid') 
    ])
    
    model.compile(
        optimizer='adam',
        loss=tf.keras.losses.BinaryCrossentropy(label_smoothing = LABEL_SMOOTHING),
        metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy'),
                 tf.keras.metrics.AUC(name='auc')])
    print(model.summary())

# for saving best model from the best epoch 
sv = tf.keras.callbacks.ModelCheckpoint(
        'cnn_best.h5', monitor='val_loss', verbose=0, save_best_only=True,
        save_weights_only=True, mode='min', save_freq='epoch')

print('#'*25)
print('#### Image Size %i with EfficientNet B%i and batch_size %i'%
      (IMG_SIZES[0],EFF_NETS,BATCH_SIZES[0]))

# TRAIN
print('Training...')         
history = model.fit(
    train_dataset, 
    epochs=EPOCHS, 
    callbacks=[sv,get_lr_callback(BATCH_SIZES[0])],     # lr schedule
    steps_per_epoch=count_data_items(files_train) // BATCH_SIZES[0],
    validation_data=load_dataset(files_valid, labeled=True)                                        
                     .cache()
                     .batch(BATCH_SIZES[0])
                     .prefetch(AUTO),                         
    verbose=VERBOSE) 

# LOAD BEST MODEL
print('Loading best model...')
model.load_weights('cnn_best.h5')    

Predict on Validation and test set using test time augmentation

In [ ]:
print('Predicting Valid with TTA...')

ct_valid = count_data_items(files_valid)
STEPS = TTA * ct_valid/BATCH_SIZES[0]/4  # number of steps to go through all TTA images
# slice to throw away images that pass the steps
pred = model.predict(ds_valid,steps=STEPS,verbose=VERBOSE)[:TTA*ct_valid,] 
# store the average of each valid image 
pred_.append(np.mean(pred.reshape((ct_valid,TTA),order='F'),axis=1)) 

# GET OOF TARGETS, FOLDS, AND NAMES
# get targets 
# do not repeat=True here as we only want the target values 
ds_valid = (load_dataset(files_valid, labeled=True, ordered=True) # do not shuffle
                    .cache()
                    .batch(BATCH_SIZES[0]*4)
                    .prefetch(AUTO))
tar_.append(np.array([target.numpy() for img, target in iter(ds_valid.unbatch())]) ) 
# get names
ds = (load_dataset(files_valid, labeled=False, return_imgname=True, ordered=True)                                        
            .cache()     
            .batch(BATCH_SIZES[0]*4)                   
            .prefetch(AUTO)) 
names_.append(np.array([img_name.numpy().decode("utf-8") for img, img_name in iter(
    ds.unbatch())]))      

# PREDICT TEST USING TTA
print('Predicting Test with TTA...')
ct_test = count_data_items(files_test)
STEPS = TTA * ct_test/BATCH_SIZES[0]/4 # number of steps to go through all TTA images
# slice to throw away images that pass the steps
pred = model.predict(ds_test,steps=STEPS,verbose=VERBOSE)[:TTA*ct_test,] 
# store the average pred of each test image
preds[:,0] += np.mean(pred.reshape((ct_test,TTA),order='F'),axis=1)

# REPORT RESULTS
auc_ = roc_auc_score(tar_[-1],pred_[-1])
val_.append(np.max(history.history['val_auc']))
print('#### AUC without TTA = %.3f, with TTA = %.3f'%(val_[-1],auc_)) 

Plot train history curve

In [ ]:
display_training_curves(history, 'effnet_train', 'EfficientNet B6 Training Curve') 

In [ ]:
# SAVE TO DISK
df = pd.DataFrame(dict(
    image_name = names_[-1], target=tar_[-1], pred = pred_[-1]))
df.to_csv('effnet.csv',index=False) 
df.head() 

#### ROC and PR curves

In [ ]:
# AUROC
fpr, tpr, _ = roc_curve(df.target, df.pred)
roc_auc = auc(fpr, tpr)

# AUPRC
precision, recall, thresholds = precision_recall_curve(df.target, df.pred)
average_precision = average_precision_score(df.target, df.pred)

# PLOT
# auroc
fig = plt.figure(figsize=(18,6))
ax1 = fig.add_subplot(121)

fig.suptitle('ROC and PRC Curves for EfficientNet B6', size=25)

ax1.plot([0, 1], [0, 1], linestyle='--', lw=4, color='r',
        label='Chance', alpha=.8)

ax1.plot(fpr, tpr, color='b',
        label=r'ROC (AUC = %0.2f)' % (roc_auc),
        lw=4, alpha=.8)    
    
ax1.set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05])
ax1.set_title("Receiver Operating Characteristic Curve", size=20)
ax1.set_xlabel('False Positive Rate',size=20); plt.xticks(size=15)
ax1.set_ylabel('True Positive Rate',size=20); plt.yticks(size=15)
ax1.legend(loc="lower right",prop={"size":15})

# auprc
ax2 = fig.add_subplot(122)
ax2.step(recall, precision, where='post', color='b',
        label=r'AP (AP = %0.2f)' % (average_precision),
        lw=4, alpha=.8)    
    
ax2.set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05])
ax2.set_title("Precision Recall Curve", size=20)
ax2.set_xlabel('Recall',size=20); plt.xticks(size=15)
ax2.set_ylabel('Precision',size=20); plt.yticks(size=15)
ax2.legend(loc="lower left",prop={"size":15})
plt.savefig('effnet_roc.png')
plt.show()

In [ ]:
# Helper function to calculate the F1 Score
def calc_f1(prec, recall):
    return 2*(prec*recall)/(prec+recall) if recall and prec else 0

In [ ]:
# Calculate the f1 score for each threshold
f1score = [calc_f1(precision[i], recall[i]) for i in range(len(thresholds))]

# Get the highest f1score
idx = np.argmax(f1score)

# Get the highest precision, recall, threshold and f1score
precision = round(precision[idx], 4)
recall = round(recall[idx], 4)
threshold = round(thresholds[idx], 4)
f1score = round(f1score[idx], 4)

print('Precision:', precision)
print('Recall:', recall)
print('Threshold:', threshold)
print('F1 Score:', f1score)

In [ ]:
# Plot a confusion matrix
binary_preds = [0 if x < threshold else 1 for x in df.pred]
cm = confusion_matrix(df.target, binary_preds)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True)
plt.xlabel('Predicted label', size=10)
plt.ylabel('True label', size=10)
plt.title('EfficientNet B6 Confusion Matrix', size=15)
plt.savefig('effnet_cm.png')
plt.show() 

#### Predict for an image

In [ ]:
IMAGE_PATH = "../input/siim-isic-melanoma-classification/jpeg/test/ISIC_0052060.jpg"
img = tf.keras.preprocessing.image.load_img(IMAGE_PATH, 
                                            target_size=(IMG_SIZES[0], IMG_SIZES[0]))
plt.imshow(img)
origin_img = img

In [ ]:
# Get the prediction for the image
prediction = model.predict(np.expand_dims(img, axis=0))
binary_prediction = [0 if x < 0.5 else 1 for x in prediction]
print("Prediction: " + ("Benign" if binary_prediction == 0 else "Malignant")) 

## 4. EffNet + BiLSTM (Hybrid)

In [ ]:
def ReshapeLayer(x):
    '''
    Reshape CNN output
    '''
    shape = x.shape 
    # H,W * channel
    reshape = Reshape((shape[1],1))(x)

    return reshape

In [ ]:
train_dataset = (load_dataset(files_train, labeled=True)
                     .repeat() # repeat to continue getting data for aug
                     .map(data_augment, num_parallel_calls=AUTO) # data augmentation
                     .shuffle(SEED)
                     .batch(BATCH_SIZES[0],drop_remainder=True)
                     # prefetch next batch while training (autotune prefetch buffer size)
                     .prefetch(AUTO)) 

ds_valid = (load_dataset(files_valid, labeled=True, ordered=True)                                        
                     .cache()     
                     .repeat()   # repeat for data aug during val
                     .map(data_augment, num_parallel_calls=AUTO)  # data augmentation 
                     .batch(BATCH_SIZES[0]*4)  # X4 to speed up training
                     .prefetch(AUTO))

ds_test = (load_dataset(files_test, labeled=False, ordered=True) # do not shuffle
                     .repeat()                                   # repeat for TTA
                     .map(data_augment, num_parallel_calls=AUTO) # data augmentation 
                     .batch(BATCH_SIZES[0]*4)
                     .prefetch(AUTO))

#### Build, Train and Evaluate model  

In [ ]:
# initialize storage
pred_ = []; tar_ = []; val_ = []; names_ = []
# initialize for prediction storage
preds = np.zeros((count_data_items(test_filenames),1))  

In [ ]:
# USE VERBOSE=0 for silent, VERBOSE=1 for interactive, VERBOSE=2 for commit
VERBOSE = 2

if tpu: tf.tpu.experimental.initialize_tpu_system(tpu)
    
K.clear_session()
with strategy.scope():
    model = tf.keras.Sequential([
        efn.EfficientNetB6(input_shape=(IMG_SIZES[0],IMG_SIZES[0], 3),
                           weights='imagenet',include_top=False),
        GlobalAveragePooling2D(),
        # reshape layer
        Lambda(ReshapeLayer),
        # add BiLSTM layer
        Bidirectional(LSTM(150, return_sequences=True, dropout=0.3)),
        Bidirectional(LSTM(96, dropout=0.3)),
        # dense layer
        # Dense(8, activation='relu'),
        # add fully connected layer, with sigmoid activation since only 2 categories
        Dense(1, activation='sigmoid') 
    ])
    
    model.compile(
        optimizer='adam',
        loss=tf.keras.losses.BinaryCrossentropy(label_smoothing = LABEL_SMOOTHING),
        metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy'),
                 tf.keras.metrics.AUC(name='auc')])
    print(model.summary())

# for saving best model from the best epoch 
sv = tf.keras.callbacks.ModelCheckpoint(
        'bi_best.h5', monitor='val_loss', verbose=0, save_best_only=True,
        save_weights_only=True, mode='min', save_freq='epoch')

print('#'*25)
print('#### Image Size %i with EfficientNet B%i + BiLSTM and batch_size %i'%
      (IMG_SIZES[0],EFF_NETS,BATCH_SIZES[0]))

# TRAIN
print('Training...')         
history = model.fit(
    train_dataset, 
    epochs=EPOCHS, 
    callbacks=[sv,get_lr_callback(BATCH_SIZES[0])],     # lr schedule
    steps_per_epoch=count_data_items(files_train) // BATCH_SIZES[0],
    validation_data=load_dataset(files_valid, labeled=True)                                        
                     .cache()
                     .batch(BATCH_SIZES[0])
                     .prefetch(AUTO),                         
    verbose=VERBOSE) 

# LOAD BEST MODEL
print('Loading best model...')
model.load_weights('bi_best.h5') 

Predict valid and test dataset using test time augmentation (TTA). 

In [ ]:
print('Predicting Valid with TTA...')

ct_valid = count_data_items(files_valid)
STEPS = TTA * ct_valid/BATCH_SIZES[0]/4  # number of steps to go through all TTA images
# slice to throw away images that pass the steps
pred = model.predict(ds_valid,steps=STEPS,verbose=VERBOSE)[:TTA*ct_valid,] 
# store the average of each valid image 
pred_.append(np.mean(pred.reshape((ct_valid,TTA),order='F'),axis=1)) 

# GET OOF TARGETS, FOLDS, AND NAMES
# get targets 
# do not repeat=True here as we only want the target values 
ds_valid = (load_dataset(files_valid, labeled=True, ordered=True) # do not shuffle
                    .cache()
                    .batch(BATCH_SIZES[0]*4)
                    .prefetch(AUTO))
tar_.append(np.array([target.numpy() for img, target in iter(ds_valid.unbatch())]) ) 
# get names
ds = (load_dataset(files_valid, labeled=False, return_imgname=True, ordered=True)                                        
            .cache()     
            .batch(BATCH_SIZES[0]*4)                   
            .prefetch(AUTO)) 
names_.append(np.array([img_name.numpy().decode("utf-8") for img, img_name in iter(
    ds.unbatch())]))      

# PREDICT TEST USING TTA
print('Predicting Test with TTA...')
ct_test = count_data_items(files_test)
STEPS = TTA * ct_test/BATCH_SIZES[0]/4 # number of steps to go through all TTA images
# slice to throw away images that pass the steps
pred = model.predict(ds_test,steps=STEPS,verbose=VERBOSE)[:TTA*ct_test,] 
# store the average pred of each test image
preds[:,0] += np.mean(pred.reshape((ct_test,TTA),order='F'),axis=1)

# REPORT RESULTS
auc_ = roc_auc_score(tar_[-1],pred_[-1])
val_.append(np.max(history.history['val_auc']))
print('#### AUC without TTA = %.3f, with TTA = %.3f'%(val_[-1],auc_)) 

In [ ]:
display_training_curves(history, 'bi_lstm_curve', 'EfficientNet B6 + BiLSTM Training Curve')

In [ ]:
# SAVE TO DISK
df = pd.DataFrame(dict(
    image_name = names_[-1], target=tar_[-1], pred = pred_[-1]))
df.to_csv('bilstm.csv',index=False) 
df.head() 

#### ROC and PR Curves

In [ ]:
from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score

# AUROC
fpr, tpr, _ = roc_curve(df.target, df.pred)
roc_auc = auc(fpr, tpr)

# AUPRC
precision, recall, thresholds = precision_recall_curve(df.target, df.pred)
average_precision = average_precision_score(df.target, df.pred)

# PLOT
# auroc
fig = plt.figure(figsize=(18,6))
ax1 = fig.add_subplot(121)

fig.suptitle('ROC and PRC Curves for EfficientNet B6 + BiLSTM', size=25)

ax1.plot([0, 1], [0, 1], linestyle='--', lw=4, color='r',
        label='Chance', alpha=.8)

ax1.plot(fpr, tpr, color='b',
        label=r'ROC (AUC = %0.2f)' % (roc_auc),
        lw=4, alpha=.8)    
    
ax1.set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05])
ax1.set_title("Receiver Operating Characteristic Curve", size=20)
ax1.set_xlabel('False Positive Rate',size=20); plt.xticks(size=15)
ax1.set_ylabel('True Positive Rate',size=20); plt.yticks(size=15)
ax1.legend(loc="lower right",prop={"size":15})

# auprc
ax2 = fig.add_subplot(122)
ax2.step(recall, precision, where='post', color='b',
        label=r'AP (AP = %0.2f)' % (average_precision),
        lw=4, alpha=.8)    
    
ax2.set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05])
ax2.set_title("Precision Recall Curve", size=20)
ax2.set_xlabel('Recall',size=20); plt.xticks(size=15)
ax2.set_ylabel('Precision',size=20); plt.yticks(size=15)
ax2.legend(loc="lower left",prop={"size":15})
plt.savefig('bilstm_roc.png')
plt.show()

In [ ]:
# Helper function to calculate the F1 Score
def calc_f1(prec, recall):
    return 2*(prec*recall)/(prec+recall) if recall and prec else 0

In [ ]:
# Calculate the f1 score for each threshold
f1score = [calc_f1(precision[i], recall[i]) for i in range(len(thresholds))]

# Get the highest f1score
idx = np.argmax(f1score)

# Get the highest precision, recall, threshold and f1score
precision = round(precision[idx], 4)
recall = round(recall[idx], 4)
threshold = round(thresholds[idx], 4)
f1score = round(f1score[idx], 4)

print('Precision:', precision)
print('Recall:', recall)
print('Threshold:', threshold)
print('F1 Score:', f1score)

In [ ]:
# Plot a confusion matrix
binary_preds = [0 if x < threshold else 1 for x in df.pred]
cm = confusion_matrix(df.target, binary_preds)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True)
plt.xlabel('Predicted label', size=10)
plt.ylabel('True label', size=10)
plt.title('EfficientNet B6 + BiLSTM Confusion Matrix', size=15)
plt.savefig('bilstm_cm.png')
plt.show() 

#### Predict for an image

In [ ]:
IMAGE_PATH = "../input/siim-isic-melanoma-classification/jpeg/test/ISIC_0052060.jpg"
img = tf.keras.preprocessing.image.load_img(IMAGE_PATH, 
                                            target_size=(IMG_SIZES[0], IMG_SIZES[0]))
plt.imshow(img)
origin_img = img 

In [ ]:
# Get the prediction for the image
prediction = model.predict(np.expand_dims(img, axis=0))
binary_prediction = [0 if x < 0.5 else 1 for x in prediction]
print("Prediction: " + ("Benign" if binary_prediction == 0 else "Malignant"))